In [1]:
import pandas as pd

predict_edges = pd.read_csv('../../data/test.txt', sep=',', header=None)
predict_edges.columns = ['source', 'target']
tfidf_embeddings = pd.read_csv('../../data/abstracts_tfidf_embeddings.csv')
tfidf_embeddings.head(2)
predict_edges_cosine_similarity_df = pd.DataFrame(columns=['source', 'target', 'cosine_similarity'])


from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm


predict_edges_cosine_similarity_df['source'] = predict_edges['source']
predict_edges_cosine_similarity_df['target'] = predict_edges['target']

cosine_similarities = []
for idx, row in tqdm(predict_edges_cosine_similarity_df.iterrows(), total=predict_edges_cosine_similarity_df.shape[0]):
    source_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['source']].iloc[:, 1:-1].values
    target_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['target']].iloc[:, 1:-1].values
    if source_vec.size > 0 and target_vec.size > 0:
        sim = cosine_similarity(source_vec, target_vec)[0][0]
    else:
        sim = None
    cosine_similarities.append(sim)

predict_edges_cosine_similarity_df['cosine_similarity'] = cosine_similarities

predict_edges_cosine_similarity_df.to_csv('../../data/training/predict/predict_edges_cosine_similarity.csv', index=False)




  0%|          | 0/106692 [00:00<?, ?it/s]